# Transaction level timelocks (nLocktime and nSequence)

In this section we'll cover how to create a transaction level timelocks with `nLocktime` and `nSequence`.

Absolute timelocks are used for protection against [fee sniping](https://bitcoinops.org/en/topics/fee-sniping/#:~:text=Fee%20sniping%20occurs%20when%20a,who%20originally%20created%20those%20blocks.). Both absolute and relative timelocks are used in layer 2 protocols such lightning HTLC transactions, cross chain atomic swaps, however in those they are used in conjunction with the OP_CODES OP_CHECKLOCKTIMEVERIFY and OP_CHECKSEQUENCEVERIFY. We will cover those op codes in the next section.

## Prerequisite knowledge
- For all notebooks:
    - A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook) by Andreas Antonopoulos UTXO model, in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
    - A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
    - [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).


- Specific to this notebook:
    - SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
    - Base58 addresses - '[Addresses chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
    - Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
    - TestShell setup - '[P2PKH chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/chapter1-legacy/p2pkh.ipynb)'

### Further Reading
- https://medium.com/summa-technology/bitcoins-time-locks-27e0c362d7a1

## Setup 

### Requirements
For this exercise we'll need Bitcoin Core. This notebook has been tested with [v24.0.1](https://github.com/bitcoin/bitcoin/releases/tag/v24.0.1).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing bitcoin-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [1]:
path_to_bitcoin_functional_test = "/Users/dariuscognac/bitcoin/test/functional"
path_to_bitcoin_tx_tutorial = "/Users/dariuscognac/Documents/Github/bitcoin-tx-tutorial"

import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_bitcoin_tx_tutorial)
from functions import *

import json

### Scenario
Kim is an old grandmother who wants to pass some of her wealth (0.1 btc) on to her grandson, Peter. However, Peter is still a child and Kim doesn't trust Peter to spend his bitcoin wisely. Kim would rather wait until Peter turns 18 before he's able to spend his bitcoin. Kim doesn't know how much longer she'll be around for, and she doesn't trust anyone else to handle her money. To give Peter bitcoin in a way which will only allow him to spend it once he's 18, Kim will do this by giving Peter a signed transaction for 0.1 btc with an absolute timelock set to expire on his 18th birthday.

Note that 1 year corresponds to about 50,000 blocks. To save us having to mine tens of thousands of regtest blocks (which may take a few minutes), we'll instead suppose Peter's 18th birthday is happening much sooner, at block 500.

### Create a P2WPKH UTXO

For this example, Kim is spending from a P2WPKH output, although the same mechanism would work with any other type of output. We'll use the same code from the P2WPKH example to create the utxo.

In [2]:
sender_privkey = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
sender_pubkey = privkey_to_pubkey(sender_privkey)
address_to_spend = pk_to_p2wpkh(sender_pubkey, network = "regtest")
print("sender's p2wpkh address: " + address_to_spend)

sender's p2wpkh address: bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw


#### Create Kim's utxo with 0.101 btc (0.001 btc is for the next tx fee)

Now that we have the address, we'll fund it using the `TestShell` commands. For more on these steps you can view the [function definitions](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/functions/setup_testshell.py) or look at the first [P2PKH notebook](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/chapter1-legacy/p2pkh.ipynb) example.

In [3]:
node = setup_testshell()
txid_to_spend, index_to_spend = fund_address(node, address_to_spend, 0.101)
print(f"txid: {txid_to_spend}, {index_to_spend}")

2022-12-08T11:20:14.091000Z TestFramework (INFO): Initializing test directory /var/folders/r5/yk8yg2xs1gs8xzkn5l8vr72w0000gn/T/bitcoin_func_test_6cipa0pr
txid: f8830ec636a360ca9dc0f267ab7f29d2675a7066a9b405c1ae9e22664ffba557, 1


#### Set the output scriptPubkey and amount

Now that we have created Kim's P2WPKH utxo, we can create her timelocked transaction spending from it. Let's say Peter's address is `bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw`, which corresponds to the scriptPubkey `0014fc7250a211deddc70ee5a2738de5f07817351cef`. We'll also set the amount to spend the entire utxo, excluding the miner fee (of 0.001 btc).

In [4]:
output1_spk = bytes.fromhex("0014fc7250a211deddc70ee5a2738de5f07817351cef")
output1_value_sat = int(float("0.1") * 100000000)

## Absolute timelocks (nLocktime)

Times are expressed as an unsigned 32 bit (4 byte) integer. If time_lock is 0, it's ignored. If it is 500,000,000 or above, it's treated as a unix timestamp. To set nLocktime, we'll use the built in python method `to_bytes` to convert the integer value for the blockheight into an unsigned 32 bit integer.

In [5]:
# let's suppose Peter's 18th birthday is estimated to happen at block 500
locktime_int = 500
locktime = locktime_int.to_bytes(4, byteorder="little", signed=False)
print("nLocktime: ", locktime.hex())

nLocktime:  f4010000


Note that `nLocktime` will be ignored if `nSequence` is set to the largest value (`0xffffffff`). Using `nSequence` to signal in this way is an artifact from Satoshi's half-baked time-lock implementation and at this point it would require a hardfork to change it. To enable `nLocktime` we'll set `nSequence` to `0xffff fffe`. Note that this value for `nSequence` is also the default set by bitcoin core.

In [6]:
sequence = bytes.fromhex("ffff fffe")
print("sequence: ", sequence.hex())

sequence:  fffffffe


### Create an unsigned transaction

Using the values for `nLocktime` that we set above `nSequence`, we'll create the unsigned transaction and sign it in the same way we did when creating a regular P2WPKH transaction.

In [7]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# SEQUENCE
# set above

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("01")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
)

# LOCKTIME
# set above

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  020000000157a5fb4f66229eaec105b4a966705a67d2297fab67f2c09dca60a336c60e83f80100000000fffffffe018096980000000000160014fc7250a211deddc70ee5a2738de5f07817351ceff4010000


### Signing the transaction

Segwit transaction signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki)


In [8]:
pk_hash = hash160(sender_pubkey)
scriptcode = bytes.fromhex("76a914" + pk_hash.hex() + "88ac")

input_amount_sat = int(0.101 * 100_000_000)
value = input_amount_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(txid + index)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + txid
    + index
    + varint_len(scriptcode)
    + scriptcode
    + value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print(tx_digest_preimage.hex())

020000006e1879543ea79c0888efd4bae2fdd50e17092fe09f62280152b5e2eb69ce37c9bbdebfac1cd12080fb29f8919f7f8be7bdb21a89c8d09e8a59fc7bb1d2a7379657a5fb4f66229eaec105b4a966705a67d2297fab67f2c09dca60a336c60e83f8010000001976a914fc7250a211deddc70ee5a2738de5f07817351cef88ac201d9a0000000000fffffffe62580c97423350cc6d13c95a8203b8dd1962f9f34ecaa00a7542aa14f6e2b1a3f401000001000000


Hash this transaction and produce an ecdsa signature on it. 

In [9]:
# Create sigHash to be signed
sighash = hash256(tx_digest_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(sender_privkey, curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Witness field
witness = (
    # indicate the number of stack items for the txin
    # 2 items for signature and pubkey
    bytes.fromhex("02")
    + varint_len(signature)
    + signature
    + varint_len(sender_pubkey)
    + sender_pubkey
)

# tx_in with our new sigScript containing the signature we just created
inputs_signed = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  0200000000010157a5fb4f66229eaec105b4a966705a67d2297fab67f2c09dca60a336c60e83f80100000000fffffffe018096980000000000160014fc7250a211deddc70ee5a2738de5f07817351cef024730440220233a739ab1642554ee0f57b6ff17082b5e7f955936ae20ba4be8587991de0f82022005b2c09aa396cd20ad0de873091e855cdec82d56253f45c08da1b886e28242eb0121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aaf4010000


### Scenario
Kim has now successfully created a transaction which can only be spent by Peter after his 18th birthday (block 500)! Note that Peter won't be able to broadcast this transaction until the block height has been reached.

If he were to try broadcasting the transaction, it would result in the following `non-final` error.

In [10]:
height = node.getblockcount()
print("Blockchain height: ", height)

result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
print(result)

Blockchain height:  102
[{'txid': '7e4c8a4f7cc87d6ce28c180257eb4a13d558629d5cb97a37ec98c39bb123d83c', 'wtxid': '93266ce4276f2aa549adeb4f293a5eb02c40e435d479608d400ff11f0ed1529a', 'allowed': False, 'reject-reason': 'non-final'}]


Now we'll simulate 398 blocks passing (to reach block 502), and see what happens if Peter tries to broadcast his transaction again.

In [11]:
node.generate(398, invalid_call=False)
height = node.getblockcount()
print("Blockchain height: ", height)

result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
print(result)

Blockchain height:  500
[{'txid': '7e4c8a4f7cc87d6ce28c180257eb4a13d558629d5cb97a37ec98c39bb123d83c', 'wtxid': '93266ce4276f2aa549adeb4f293a5eb02c40e435d479608d400ff11f0ed1529a', 'allowed': True, 'vsize': 110, 'fees': {'base': Decimal('0.00100000')}}]


This time the transaction is successfully broadcasted and Peter has full ownership of his gift. 

To view the decoded transaction, uncomment the lines below.

In [12]:
# decoded = node.decoderawtransaction(unsigned_tx.hex())
# print(json.dumps(decoded, indent=2, default=str))

## Relative timelocks (nSequence)

Relative timelocks allow the user to set a delay from when the input was confirmed. Note that this means each input has it's own `nSequence` field which must be satisfied for the transaction to be considered 'final'.

In the scenario we just covered, Kim could have achieved the same outcome but using a relative timelock instead. The only difference would be that the delay would be specified relative to when the input was confirmed, rather than the absolute block height. Note however, that if anything happens to Kim's input (e.g. a reorg changes the block height of the input), this would affect the transaction using `nSequence`, but not the one using `nLocktime`. 

Note that `version` must be set to **`2`** for relative timelocks to be enabled.

### Exercise
Repeat the scenario above, but setting `nLocktime` to 0 instead, and relying on `nSequence`.

Hint: the block chain starts off at block 102, so the relative timelock should be set to 398 blocks for the transaction to be broadcastable at block 500.

Hint 2: nSequence is a 32 bit unsigned integer, so you can use the same `.to_bytes` method used to convert `locktime_int` to `locktime` in the previous example.


#### Solution for running with nSequence
```
# locktime_int = 0
# locktime = locktime_int.to_bytes(4, byteorder="little", signed=False)

# sequence_int = 398
# sequence = sequence_int.to_bytes(4, byteorder="little", signed=False)
# print("sequence: ", sequence.hex())
```

## Quiz

- 1. What would happen if Kim had created the transaction with `nSequence` set to `0xffffffff`?
- 2. In Peter guaranteed to be able to broadcast the timelocked transaction (signed by Kim) after block 500?
- 3. What would happen if you tried using a relative timelock with `version` set to `0x01000000`?
- 4. Is it possible to use an absolute timelock and a relative timelock in the same transaction?

## Answers
- 1. nLocktime would not have been enforced, meaning Peter would have been able to broadcast the transaction immediately.
- 2. No. Kim may create a new transaction that spends the input before Peter has a chance to broadcast his transaction.
- 3. Version 1 transactions do not enforce relative timelocks. The node would therefore ignore the relative timelock and allow it to be broadcast.
- 4. Yes.

## Exercise
- Try creating a timelock using the unix timestamp instead of the blockheight. Note that the timestamp in regtest blocks is taken from the system's clock. Also, since [BIP-113](https://github.com/bitcoin/bips/blob/master/bip-0113.mediawiki), time-locked transactions are compared to the median of the last 11 blocks.